In [1]:
import torch
import torch.nn as nn

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [3]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [4]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [6]:
class TransformerBlock( nn.Module ):
    def __init__( self, cfg ):
        super().__init__()
        self.norm1 = LayerNorm( cfg["emb_dim"] )
        self.att = MultiHeadAttention( 
            d_in = cfg["emb_dim"], 
            d_out = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            dropout = cfg["drop_rate"], 
            num_heads = cfg["n_heads"] 
        )
        self.dropout = nn.Dropout( cfg["drop_rate"] )
        self.norm2 = LayerNorm( cfg["emb_dim"] )
        self.feedforward = FeedForward( cfg )
        
    def forward( self, x ):
        
        # the attention chunk in this transformer:
        shortcut = x 
        x = self.norm1( x )
        x = self.att( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        # the feed forward chunk:
        shortcut = x
        x = self.norm2( x ) 
        x = self.feedforward( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        return x 

In [7]:
# instantiate TransformerBlock
block = TransformerBlock( GPT_CONFIG_124M )

In [8]:
input = torch.randn( 3, 4, 768 )

In [9]:
input

tensor([[[ 0.5664,  1.0047, -0.1563,  ..., -0.0531,  0.8751, -1.1452],
         [-1.0083,  1.1750, -0.5079,  ..., -1.2880, -2.6997,  0.9886],
         [-0.1540,  1.2364,  1.2237,  ...,  2.2748,  1.4736,  0.0436],
         [-0.7458,  0.3479, -0.1390,  ...,  1.2876,  0.4220,  0.4467]],

        [[ 0.5971,  0.7516,  0.2045,  ..., -0.7638,  0.1742, -0.5018],
         [ 0.1285, -0.4869, -0.5565,  ..., -0.1840,  1.0958, -0.6777],
         [ 0.6784,  0.0542, -0.6195,  ...,  0.2615,  0.1433,  0.3136],
         [ 0.1468,  0.7433, -1.2779,  ...,  0.4641,  0.5044,  0.9717]],

        [[ 0.6151,  0.6213,  0.6667,  ...,  0.2962, -1.1402, -0.2173],
         [ 0.9497,  0.0474,  0.9230,  ..., -1.9781,  0.5344, -0.3135],
         [-0.5051,  0.2362,  0.8526,  ...,  0.2474,  1.3379,  1.3115],
         [-1.8358,  3.0303, -0.9361,  ...,  0.0153,  0.8298, -0.9526]]])

In [10]:
output = block( input )

In [11]:
output.shape

torch.Size([3, 4, 768])

In [12]:
output

tensor([[[ 3.5608e-01,  1.1816e+00,  7.7872e-03,  ...,  5.7443e-01,
           1.1342e+00, -5.8779e-01],
         [-1.3468e+00,  1.3791e+00, -5.2145e-01,  ..., -1.2163e+00,
          -2.7602e+00,  1.6622e+00],
         [-7.3926e-01,  1.2197e+00,  1.0502e+00,  ...,  2.5756e+00,
           1.3942e+00,  7.8925e-02],
         [-5.5390e-01,  3.8338e-01, -3.4060e-01,  ...,  1.2447e+00,
           3.6847e-01,  2.3194e-01]],

        [[ 5.9710e-01,  5.8837e-01, -2.3331e-01,  ..., -1.2736e+00,
           9.9817e-02, -1.3823e-01],
         [ 7.9225e-02, -8.6939e-01, -1.2442e+00,  ..., -5.0428e-01,
           1.2816e+00, -3.2902e-01],
         [ 6.3527e-01, -9.0860e-01, -9.4643e-01,  ...,  4.2167e-01,
           7.9321e-03,  5.0565e-01],
         [ 2.4084e-01,  3.4536e-01, -1.4148e+00,  ...,  3.2380e-01,
           8.6806e-01,  1.0673e+00]],

        [[ 1.2394e+00,  5.2244e-01, -5.7697e-04,  ...,  1.0942e-01,
          -8.9437e-01,  1.6695e-01],
         [ 1.2596e+00,  1.2546e-01,  7.7201e-01,  .